In [ ]:
# Importing all the required libraries in python
import os
import torch
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
import shutil as sh

import matplotlib.pyplot as plt
from IPython.display import Image, clear_output
%matplotlib inline

In [ ]:
# Reading the dataset from train.csv file
df = pd.read_csv('../input/global-wheat-detection/train.csv')
df.head()

In [ ]:
# Read the dataset

df = pd.read_csv('../input/global-wheat-detection/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = (df['x'] + df['w'])/2
df['y_center'] = (df['y'] + df['h'])/2
df['classes'] = 0

# Making new dataset in the same format YOLOv5 needed
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]
df.head()

In [ ]:
index = list(set(df.image_id))
print("Total Images: ",len(index))

In [ ]:
# This cell will automatically make the dataset and save it to convertor folder
source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val/'
            else:
                path2save = 'train/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            with open('convertor/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,name),'convertor/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))
    

In [ ]:
# Cloning the repo
!git clone https://github.com/ultralytics/yolov5.git
clear_output()

In [ ]:
# Moving the folders to our working directory
!mv ./yolov5/* ./

In [ ]:
# Checking if the files correctly cloned and moved
!ls

In [ ]:
# installing the requirements file
!pip install -r requirements.txt
clear_output()

In [ ]:
#customize iPython writefile so we can write variables

from IPython.core.magic import register_line_cell_magic
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
# Directories under Convertor folder

print(os.listdir("./convertor/fold0"))

In [ ]:
# Making a directory for storing our data.yaml and custom YOLOv5(..).yml files
!mkdir DataFile

In [ ]:
%%writetemplate ./DataFile/data.yaml

# training directory
train: ./convertor/fold0/images/train
    
# validation directory
val: ./convertor/fold0/images/val

# I'll use validation directory for test image
test: ./convertor/fold0/images/val

# number of class
nc: 1

# name of the class
names: ['Wheat'] 

Our models structure files are save here. These are used for training [coco dataset](https://cocodataset.org/#home). But for using for our custom dataset, we have to change the nc parameter to 1, insted of 80.

In [ ]:
print(os.listdir("./models"))

In [ ]:
# checking the yolov5s model architecture
!cat ./models/yolov5.yaml

Modifying the yolov5s model architecture for nc: 1

In [ ]:
%%writetemplate ./DataFile/customYOLOv5x.yaml

# parameters
nc: 80  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# # You should skip this line
# !wandb off

In [ ]:
%%time
!python train.py --img 640  --batch 16 --epochs 1 --data ./DataFile/data.yaml --cfg ./DataFile/customYOLOv5x.yaml --weights yolov5x.pt  --name Result --cache

# Training is completed.
Results are saved in ./runs/train directory

In [ ]:
# let's check the training result directory.
# Here model and result are saved
!ls -R ./runs/train

In [ ]:
# Analize the training and validation result
Image('./runs/train/Result/results.png')

In [ ]:
# Analize the Confusion matrix
Image('./runs/train/Result/confusion_matrix.png',width=800)

In [ ]:
!python detect.py --img-size 800  --conf 0.2 --source ../input/global-wheat-detection/test --weights ./runs/train/Result/weights/best.pt --augment 

In [ ]:
# Detected images/video are saved in this path
!ls -R runs/detect

In [ ]:
Image('runs/detect/exp/2fd875eaa.jpg',width=600)